# Backward Error Analysis

## Objective

Understand backward error and backward stability:
- Forward vs backward error
- Backward stability definition
- Empirical backward error measurement
- Conditioning vs stability distinction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

from utils.linear_algebra_utils import (
    gaussian_elimination_with_pivoting,
    solve_normal_equations, solve_qr,
    create_ill_conditioned_matrix
)
from utils.error_metrics import forward_error, backward_error, condition_number

np.random.seed(42)
plt.rcParams["figure.figsize"] = (12, 6)

## Forward vs Backward Error

**Forward error**: $\|\hat{x} - x\|$ (error in output)

**Backward error**: Smallest $\Delta A, \Delta b$ such that $(A + \Delta A)\hat{x} = b + \Delta b$

**Relationship**: Forward error ≤ (condition number) × (backward error)

**Backward stability**: Algorithm produces exact solution to nearby problem

In [ ]:
# Demonstrate forward vs backward error
n = 20
condition_numbers_test = np.logspace(1, 12, 20)

forward_errors = []
backward_errors = []
cond_times_backward = []

for kappa in condition_numbers_test:
    A = create_ill_conditioned_matrix(n, kappa)
    x_true = np.random.randn(n)
    b = A @ x_true
    
    x_computed = gaussian_elimination_with_pivoting(A.copy(), b.copy())
    
    # Forward error
    fwd_err = np.linalg.norm(x_computed - x_true) / np.linalg.norm(x_true)
    
    # Backward error
    bwd_err = backward_error(A, b, x_computed)
    
    # Condition number
    cond = np.linalg.cond(A)
    
    forward_errors.append(fwd_err)
    backward_errors.append(bwd_err)
    cond_times_backward.append(cond * bwd_err)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.loglog(condition_numbers_test, forward_errors, "o-", label="Forward error", linewidth=2)
plt.loglog(condition_numbers_test, backward_errors, "s-", label="Backward error", linewidth=2)
plt.loglog(condition_numbers_test, cond_times_backward, "^-", label=r"$\kappa \times$ backward", linewidth=2)
eps = np.finfo(np.float64).eps
plt.axhline(eps, color="k", linestyle="--", label=r"$\varepsilon$", alpha=0.5)
plt.xlabel("Condition number")
plt.ylabel("Error")
plt.title("Forward vs Backward Error")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
ratio = np.array(forward_errors) / np.array(cond_times_backward)
plt.semilogx(condition_numbers_test, ratio, "o-", linewidth=2)
plt.axhline(1.0, color="k", linestyle="--", alpha=0.5)
plt.xlabel("Condition number")
plt.ylabel("Forward / (κ × Backward)")
plt.title("Verification of Error Bound")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("../plots/05_forward_vs_backward.png", dpi=150, bbox_inches="tight")
plt.show()

print("Backward error remains O(ε) - algorithm is backward stable!")
print(f"Mean backward error: {np.mean(backward_errors):.2e}")
print(f"Machine epsilon: {eps:.2e}")

## Experiment 2: Backward Stability of Different Algorithms

In [ ]:
# Compare backward errors of different methods
m, n = 100, 20
A = np.random.randn(m, n)
x_true = np.random.randn(n)
b = A @ x_true

x_normal = solve_normal_equations(A, b)
x_qr = solve_qr(A, b)

# Compute backward errors
bwd_normal = backward_error(A, b, x_normal)
bwd_qr = backward_error(A, b, x_qr)

print("Backward Error Comparison (Least Squares)")
print("=" * 50)
print(f"Normal equations: {bwd_normal:.6e}")
print(f"QR factorization: {bwd_qr:.6e}")
print(f"Machine epsilon:  {eps:.6e}")
print(f"\nBoth are backward stable (error ≈ ε)!")

## Conditioning vs Stability

**Conditioning**: Property of the **problem**
- Measures sensitivity to input perturbations
- Intrinsic to the mathematical problem
- Cannot be improved by better algorithms

**Stability**: Property of the **algorithm**
- Measures error propagation in computation
- Can be improved by better algorithms
- Backward stable algorithms are ideal

**Key insight**: Even a stable algorithm on an ill-conditioned problem produces large errors!

In [ ]:
# Demonstrate the distinction
print("Conditioning vs Stability")
print("=" * 70)
print("\nScenario 1: Stable algorithm + well-conditioned problem")
A1 = np.random.randn(10, 10)
b1 = np.random.randn(10)
x1 = gaussian_elimination_with_pivoting(A1, b1)
print(f"  Condition number: {np.linalg.cond(A1):.2e}")
print(f"  Backward error: {backward_error(A1, b1, x1):.2e}")
print(f"  Result: Small error (good!)")

print("\nScenario 2: Stable algorithm + ill-conditioned problem")
A2 = create_ill_conditioned_matrix(10, 1e12)
x_true2 = np.ones(10)
b2 = A2 @ x_true2
x2 = gaussian_elimination_with_pivoting(A2, b2)
print(f"  Condition number: {np.linalg.cond(A2):.2e}")
print(f"  Backward error: {backward_error(A2, b2, x2):.2e}")
print(f"  Forward error: {np.linalg.norm(x2 - x_true2) / np.linalg.norm(x_true2):.2e}")
print(f"  Result: Large forward error (unavoidable!)")

print("\nConclusion: Backward stability guarantees backward error ≈ ε,")
print("but forward error is bounded by κ(A) × ε")

## Key Takeaways

1. **Backward error** measures how much we perturb the problem
2. **Backward stable** algorithms have backward error ≈ ε
3. **Forward error ≤ κ(A) × backward error** (fundamental bound)
4. **Conditioning** (problem) vs **stability** (algorithm) are distinct
5. **No algorithm** can overcome ill-conditioning
6. **Best we can do**: Backward stable algorithm (solves nearby problem exactly)